## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-28-11-44-53 +0000,nypost,Minneapolis mass shooter Robin Westman’s pro-t...,https://nypost.com/2025/08/28/us-news/minneapo...
1,2025-08-28-11-40-52 +0000,wapo,India braces as 50 percent U.S. tariffs come i...,https://www.washingtonpost.com/world/2025/08/2...
2,2025-08-28-11-37-52 +0000,bbc,"Electric cars eligible for £3,750 discount ann...",https://www.bbc.com/news/articles/cvgvywyev1do...
3,2025-08-28-11-37-45 +0000,wapo,Europe furious as deadly Russian attack damage...,https://www.washingtonpost.com/world/2025/08/2...
4,2025-08-28-11-36-28 +0000,bbc,European leaders outraged after Russian strike...,https://www.bbc.com/news/articles/cvg3y7m2gz0o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2377/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
69,trump,56
0,minneapolis,17
63,shooting,17
62,school,17
265,new,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
136,2025-08-28-01-00-00 +0000,wsj,"In his first term, administration officials re...",https://www.wsj.com/politics/policy/in-trumps-...,149
109,2025-08-28-03-32-00 +0000,wsj,President Trump on Wednesday fired Robert Prim...,https://www.wsj.com/politics/policy/trump-fire...,111
275,2025-08-27-16-15-57 +0000,nypost,Cracker Barrel changed logo after call with Wh...,https://nypost.com/2025/08/27/business/cracker...,106
13,2025-08-28-11-07-19 +0000,nypost,Melania Trump ‘shattered’ by ‘senseless and de...,https://nypost.com/2025/08/28/us-news/melania-...,103
312,2025-08-27-14-30-00 +0000,wsj,A Beijing-linked espionage campaign that hit U...,https://www.wsj.com/politics/national-security...,101


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
136,149,2025-08-28-01-00-00 +0000,wsj,"In his first term, administration officials re...",https://www.wsj.com/politics/policy/in-trumps-...
110,80,2025-08-28-03-19-08 +0000,nypost,Russian ballistic missiles hit Kyiv in large-s...,https://nypost.com/2025/08/27/world-news/russi...
75,76,2025-08-28-08-06-34 +0000,bbc,FBI investigates Minneapolis school shooting a...,https://www.bbc.com/news/articles/c7760x87r4po...
109,48,2025-08-28-03-32-00 +0000,wsj,President Trump on Wednesday fired Robert Prim...,https://www.wsj.com/politics/policy/trump-fire...
251,48,2025-08-27-17-56-38 +0000,latimes,"Cal Fire captain fatally shot his girlfriend, ...",https://www.latimes.com/california/story/2025-...
275,41,2025-08-27-16-15-57 +0000,nypost,Cracker Barrel changed logo after call with Wh...,https://nypost.com/2025/08/27/business/cracker...
189,39,2025-08-27-22-03-43 +0000,cbc,CDC director Susan Monarez is being ousted les...,https://www.cbc.ca/news/world/cdc-director-sus...
124,34,2025-08-28-02-12-00 +0000,wsj,Denmark Summons U.S. Envoy Over Suspected Amer...,https://www.wsj.com/world/suspected-american-i...
248,34,2025-08-27-18-21-00 +0000,wsj,President Trump’s tariff rollout has taken num...,https://www.wsj.com/economy/trade/trump-tariff...
92,33,2025-08-28-05-02-43 +0000,bbc,North Korea's Kim Jong Un to join Putin at Chi...,https://www.bbc.com/news/articles/c17njr88ge4o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
